<span style="font-size:48px;">import library</span>


In [18]:
import pandas as pd
import json
import csv
import requests
from rembg import remove
from PIL import Image
import os
import io
from dotenv import load_dotenv
from supabase import create_client, Client

<span style="font-size:48px;">Clean ข้อมูล</span>

In [31]:
# โหลดไฟล์ CSV
file_path = "perfumes.csv"
df = pd.read_csv(file_path)

# ฟังก์ชันแปลงคอลัมน์ให้เป็น JSON array
def convert_to_json_array(column):
    df[column] = df[column].apply(
        lambda x: json.dumps([str(item.strip()) for item in x.split(",")]) if pd.notna(x) else pd.NA
    )

# คอลัมน์ที่ต้องการแปลง
columns_to_convert = ["accords", "top_notes", "middle_notes", "base_notes", "img"]

# แปลงทุกคอลัมน์ใน columns_to_convert ให้เป็น JSON array
for column in columns_to_convert:
    convert_to_json_array(column)

# บันทึกไฟล์ CSV ที่ปรับเปลี่ยนแล้ว
output_path = "data/temp/temp.csv"
df.to_csv(output_path, index=False)

print(f"บันทึกไฟล์ CSV ใหม่เรียบร้อยที่: {output_path}")


บันทึกไฟล์ CSV ใหม่เรียบร้อยที่: data/temp/temp.csv


<span style="font-size:48px;">แปลงข้อมูลตัวเลขที่เป็น string อยู่เป็น int และ float</span>

In [32]:
import pandas as pd

# โหลดไฟล์ CSV
file_path = "data/temp/temp.csv"
df = pd.read_csv(file_path)

# ฟังก์ชันแปลงค่าคอลัมน์ให้เป็นตัวเลข
def convert_to_numeric(column, is_float=False):
    # แปลงค่าเป็นตัวเลข หากไม่สามารถแปลงได้ให้เป็น NaN
    df[column] = pd.to_numeric(df[column], errors='coerce')

    if is_float:
        # แปลงเป็น float
        df[column] = df[column].astype(float)
    else:
        # แทนค่าที่เป็น NaN ด้วย NaT และแปลงเป็น int
        df[column] = df[column].where(df[column].notna(), pd.NA)
        df[column] = df[column].fillna(pd.NA).astype("Int64")  # ใช้ Int64 เพื่อล็อค NULL ได้

# แปลงค่าคอลัมน์ rating เป็น float
convert_to_numeric("rating", is_float=True)

# แปลงค่าคอลัมน์ total_votes เป็น integer โดยถ้าไม่สามารถแปลงได้ให้เป็น NaN
convert_to_numeric("total_votes", is_float=False)

# บันทึกไฟล์ CSV ที่ปรับเปลี่ยนแล้ว (ไม่มีค่าใดใน cell แทน NaN)
output_path = "data/clean/clean_perfumes_data.csv"
df.to_csv(output_path, index=False)

print(f"บันทึกไฟล์ CSV ใหม่เรียบร้อยที่: {output_path}")


บันทึกไฟล์ CSV ใหม่เรียบร้อยที่: data/clean/clean_perfumes_data.csv


<span style="font-size:48px;">โหลดรูปจาก URL</span>

In [34]:
def download_image(url, save_directory):
    # ดึงชื่อไฟล์จาก URL
    filename = os.path.basename(url)
    save_path = os.path.join(save_directory, filename)

    try:
        # ดาวน์โหลดภาพ
        response = requests.get(url, timeout=10)  # กำหนด timeout เพื่อลดความล่าช้า
        if response.status_code == 200:
            with open(save_path, 'wb') as file:
                file.write(response.content)
            print(f"Image successfully downloaded: {save_path}")
            return save_path  # คืนค่าพาธที่บันทึกไฟล์
        else:
            print(f"Failed to retrieve image from {url}")
            return None
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return None

def process_images(input_csv, output_csv, save_directory):
    # สร้างโฟลเดอร์ที่บันทึกรูปภาพ
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    # อ่าน CSV ข้อมูลเดิม
    with open(input_csv, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        rows = list(reader)

    # สร้างไฟล์ CSV ใหม่ที่เก็บตำแหน่งไฟล์ของภาพที่ดาวน์โหลด
    with open(output_csv, mode='w', newline='', encoding='utf-8') as outfile:
        fieldnames = list(rows[0].keys()) + ['img_path']  # เพิ่มคอลัมน์ 'img_path'
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        # วนลูปผ่านข้อมูลแต่ละแถวใน CSV เดิม
        for row in rows:
            img_url = row['logo']  # ดึง URL ของภาพ
            img_path = download_image(img_url, save_directory)  # ดาวน์โหลดภาพ

            if img_path:
                row['img_path'] = img_path  # เก็บพาธของไฟล์ที่ดาวน์โหลด
            else:
                row['img_path'] = None  # ถ้าดาวน์โหลดไม่สำเร็จใส่ None

            # เขียนแถวข้อมูลใหม่ลงในไฟล์ CSV
            writer.writerow(row)

    print(f"Processed data saved to {output_csv}")

# ตัวอย่างการใช้งาน
input_csv = 'perfumes.csv'  # ไฟล์ CSV ที่มีข้อมูลรูปภาพ
output_csv = 'img/downloads/logo_path.csv'  # ไฟล์ CSV ที่เก็บตำแหน่งไฟล์รูปภาพที่ดาวน์โหลด
save_directory = 'img/downloads/logo'  # โฟลเดอร์ที่ใช้บันทึกภาพ
process_images(input_csv, output_csv, save_directory)


Image successfully downloaded: img/downloads/logo\m.2087.jpg
Image successfully downloaded: img/downloads/logo\m.2087.jpg
Image successfully downloaded: img/downloads/logo\m.2087.jpg
Image successfully downloaded: img/downloads/logo\m.2087.jpg
Image successfully downloaded: img/downloads/logo\m.2087.jpg
Image successfully downloaded: img/downloads/logo\m.2087.jpg
Image successfully downloaded: img/downloads/logo\m.1939.jpg
Image successfully downloaded: img/downloads/logo\m.2278.jpg
Image successfully downloaded: img/downloads/logo\m.1939.jpg
Image successfully downloaded: img/downloads/logo\m.1939.jpg
Image successfully downloaded: img/downloads/logo\m.2087.jpg
Image successfully downloaded: img/downloads/logo\m.1939.jpg
Image successfully downloaded: img/downloads/logo\m.1939.jpg
Image successfully downloaded: img/downloads/logo\m.1939.jpg
Image successfully downloaded: img/downloads/logo\m.1939.jpg
Image successfully downloaded: img/downloads/logo\m.1939.jpg
Image successfully downl

<span style="font-size:48px;">ตัด Bg รูป</span>


In [ ]:
def remove_background(input_image_path, output_image_path):
    # เปิดภาพต้นฉบับ
    input_image = Image.open(input_image_path)

    # ตรวจสอบชนิดของไฟล์ (JPG, PNG, หรืออื่นๆ)
    input_format = input_image.format
    print(f"Input image format: {input_format}")

    # แปลงภาพเป็น PNG (รองรับโปร่งใส) หากไม่ใช่ PNG
    if input_format != 'PNG':
        input_image = input_image.convert('RGBA')  # แปลงเป็น RGBA เพื่อรองรับโปร่งใส

    # สร้างโฟลเดอร์ output หากยังไม่มี
    output_directory = os.path.dirname(output_image_path)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # บันทึกภาพเป็น PNG ชั่วคราว
    temp_input_path = 'img/temp/temp_image.png'
    input_image.save(temp_input_path)

    # อ่านภาพที่แปลงเป็น PNG
    with open(temp_input_path, 'rb') as input_file:
        input_data = input_file.read()

    # ตัดพื้นหลังออกจากภาพ
    output_data = remove(input_data)

    # แปลงข้อมูลที่ได้กลับเป็นภาพ
    output_image = Image.open(io.BytesIO(output_data))

    # บันทึกผลลัพธ์ที่ได้ในรูปแบบ PNG (รองรับโปร่งใส)
    output_image.save(output_image_path, format='PNG', optimize=True)

    # ลบไฟล์ชั่วคราวออกหลังจากเสร็จ
    os.remove(temp_input_path)

    print(f"Background removed and saved as {output_image_path}")

def process_images_from_csv(csv_file, output_directory):
    # อ่านไฟล์ CSV
    with open(csv_file, mode='r') as file:
        csv_reader = csv.reader(file)
        
        # ข้ามแถวหัวตาราง
        next(csv_reader)
        
        # สำหรับแต่ละแถวใน CSV
        for row in csv_reader:
            # ดึง path ของภาพจากคอลัมน์ img (ให้ระบุแค่เส้นทางใน CSV เช่น img/downloads/375x500.68001.jpg)
            img_path = row[0]
            
            # สร้างเส้นทางเต็มของไฟล์ภาพ
            input_image_path = os.path.join(img_path)  # เนื่องจากมีข้อมูล path อยู่แล้วในไฟล์ CSV

            # สร้างเส้นทางการบันทึกผลลัพธ์ที่ตัดพื้นหลังแล้ว
            filename = os.path.basename(img_path).split('.')[0]+(".")+os.path.basename(img_path).split('.')[1] + '.png'  # เปลี่ยนชื่อไฟล์เป็น .png
            output_image_path = os.path.join(output_directory, filename)
            
            # เรียกฟังก์ชัน remove_background เพื่อทำการตัดพื้นหลัง
            remove_background(input_image_path, output_image_path)

# ตัวอย่างการใช้งาน
csv_file = 'img/downloads/logo_path.csv'  # ระบุเส้นทางไฟล์ CSV ที่มีข้อมูล img_path
output_directory = 'img/cutbg/logo'  # โฟลเดอร์ที่ต้องการบันทึกผลลัพธ์
process_images_from_csv(csv_file, output_directory)

<span style="font-size:48px;">ปรับแก้ Img path ใหม่</span>

In [44]:
import csv

def update_image_url(input_csv, output_csv, base_url):
    # อ่านไฟล์ CSV เดิม
    with open(input_csv, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        rows = list(reader)
    
    # สร้างไฟล์ CSV ใหม่ที่เก็บ URL ที่ถูกอัปเดต
    with open(output_csv, mode='w', newline='', encoding='utf-8') as outfile:
        fieldnames = list(rows[0].keys())  # ใช้คอลัมน์เดิมจากไฟล์ CSV
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        # วนลูปผ่านแถวใน CSV และอัปเดต URL
        for row in rows:
            img_url = row['img']  # ดึง URL ของภาพจากคอลัมน์ 'img'
            if img_url:
                # แปลงจาก array เป็น string โดยตัดเครื่องหมาย "[]"
                img_url = img_url.strip('[]').replace('"', '')  # เอาเครื่องหมาย [ ] และ " ออก
                # เปลี่ยน URL ให้ตรงตามรูปแบบที่ต้องการ
                img_filename = img_url.split('/')[-1]  # ดึงชื่อไฟล์จาก URL เดิม
                img_filename_without_extension = img_filename.split('.')[0] + (".") + img_filename.split('.')[1]  # เอาชื่อไฟล์โดยไม่รวมส่วนของนามสกุล
                new_filename = f"{img_filename_without_extension}.png"  # เปลี่ยนให้นามสกุลเป็น .png
                new_url = f"{base_url}{new_filename}"  # สร้าง URL ใหม่

                row['img'] = f'["{new_url}"]' # อัปเดตคอลัมน์ 'img' ด้วย URL ใหม่


            # เขียนแถวข้อมูลใหม่ลงในไฟล์ CSV
            writer.writerow(row)

    print(f"Processed data saved to {output_csv}")

# ตัวอย่างการใช้งาน
input_csv = 'data/clean/clean_perfumes_data.csv'  # ไฟล์ CSV ที่มีข้อมูลรูปภาพ
output_csv = 'data/finish/perfumes.csv'  # ไฟล์ CSV ใหม่ที่เก็บ URL ที่อัปเดต
base_url = "https://lalzmarjvjsqavjsuiux.supabase.co/storage/v1/object/public/IMAGES/Perfume_Img/"
update_image_url(input_csv, output_csv, base_url)


Processed data saved to data/finish/perfumes.csv
